In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [12]:
import pandas as pd
import numpy as np

from src.features import build_features

import glob

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]
COLUMNAS_POBLACION_FONASA = ["2018", "2019", "2020", "2021", "2022"]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

COMUNAS_SSMC = ["Santiago", "Maipú", "Estación Central", "Cerrillos"]

## Obtener poblacion INE

In [3]:
ruta_base_de_datos = (
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
)

# Lee la base de datos
df = pd.read_excel(ruta_base_de_datos).iloc[:-2]

# Elimina el sufijo 'Poblacion' de las columnas
df.columns = df.columns.str.replace("Poblacion ", "")

# Renombra columna de hombres o mujeres
df = df.rename(columns={"Sexo\n1=Hombre\n2=Mujer": "hombre_mujer"})

# Indica si es adulto o infantil
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

## Obtener poblacion FONASA

In [4]:
archivos = glob.glob("../data/raw/5.0-poblacion-fonasa/*.csv")
df_fonasa = pd.concat(pd.read_csv(archivo, encoding="latin-1") for archivo in archivos)

df_fonasa["ANO_INFORMACION"] = df_fonasa.MES_INFORMACION.astype(str).str[:4]

df_fonasa["REGION"] = df_fonasa["REGION"].str.upper().str.strip()
df_fonasa["REGION"] = df_fonasa["REGION"].replace(
    {
        "ÑUBLE": "DE ÑUBLE",
        "DEL LIBERTADOR B. O'HIGGINS": "DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS",
    }
)

df_fonasa["SEXO"] = df_fonasa["SEXO"].str.upper().str.strip()
df_fonasa["SERVICIO_SALUD"] = df_fonasa["SERVICIO_SALUD"].str.upper().str.strip()
df_fonasa["COMUNA"] = df_fonasa["COMUNA"].str.upper().str.strip()

df_fonasa["EDAD_TRAMO"] = df_fonasa["EDAD_TRAMO"].replace(
    {"Más de 99 años": "99 años", "S.I.": "-1"}
)
df_fonasa["EDAD_TRAMO"] = df_fonasa["EDAD_TRAMO"].str.split().str[0].astype(int)

## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO
4. SSMC

### INE

In [5]:
ESTRATOS_A_CALCULAR_INE = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR_INE["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR_INE[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR_INE["SSMO"] = df_ssmo

df_ssmc = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMC)]
ESTRATOS_A_CALCULAR_INE["SSMC"] = df_ssmc

### FONASA

In [6]:
# Obtiene estratos a calcular FONASA
ESTRATOS_A_CALCULAR_FONASA = {}

# Estrato pais
df_pais_fonasa = df_fonasa.copy()
ESTRATOS_A_CALCULAR_FONASA["Pais"] = df_pais_fonasa

# Por region
for region in df_pais_fonasa["REGION"].unique():
    df_region_fonasa = df_pais_fonasa.query("REGION == @region").copy()
    ESTRATOS_A_CALCULAR_FONASA[region] = df_region_fonasa

# Obtiene SSMO y SSMC
df_ssmo_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO ORIENTE'")
df_ssmc_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO CENTRAL'")

ESTRATOS_A_CALCULAR_FONASA["SSMO"] = df_ssmo_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMC"] = df_ssmc_fonasa

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.

### INE

In [7]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "", # Todo el pais
    "hombres": "hombre_mujer == 1", # Hombres
    "mujeres": "hombre_mujer == 2", # Mujeres
    "recien_nacidos_vivos": "Edad == 0", # Recien Nacidos Vivos
    "sobre_15": "Edad > 15", # Sobre 15 anios
    "sobre_20": "Edad > 20", # Sobre 20 anios
    "sobre_45": "Edad > 45", # Sobre 45 anios
    "sobre_60": "Edad > 60", # Sobre 60 anios
    "sobre_65": "Edad > 65", # Sobre 65 anios
    "sobre_75": "Edad > 75", # Sobre 75 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59", # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74", # Entre 60 a 74 anios
    "mayores_a_75_anios": "Edad >= 75", # Mayores a 75 anios

}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.

In [8]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados_ine = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR_INE, query_strings, COLUMNAS_POBLACION_INE
)
# Une todos los resultados en un unico DataFrame
poblaciones_estratos_calculados_ine = pd.concat(poblaciones_estratos_calculados_ine)
poblaciones_estratos_calculados_ine = poblaciones_estratos_calculados_ine.reset_index(
    names=["Edad Incidencia", "Estrato"]
)

### FONASA

In [9]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "", # Todo el pais
    "hombres": "SEXO == 'HOMBRE'", # Hombres
    "mujeres": "SEXO == 'MUJER'", # Mujeres
    "recien_nacidos_vivos": "", # Recien Nacidos Vivos
    "sobre_15": "EDAD_TRAMO >= 15", # Sobre 15 anios
    "sobre_20": "EDAD_TRAMO >= 20", # Sobre 20 anios
    "sobre_45": "EDAD_TRAMO >= 45", # Sobre 45 anios
    "sobre_60": "EDAD_TRAMO >= 60", # Sobre 60 anios
    "sobre_65": "EDAD_TRAMO >= 65", # Sobre 65 anios
    "sobre_75": "EDAD_TRAMO >= 75", # Sobre 75 anios
    "entre_15_a_24_anios": "EDAD_TRAMO >= 15 and EDAD_TRAMO < 25", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "EDAD_TRAMO >= 25 and EDAD_TRAMO < 45", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "EDAD_TRAMO >= 45 and EDAD_TRAMO < 60", # Entre 45 y 59 anios
    "entre_60_y_70_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO < 75", # Entre 60 a 74 anios
    "mayores_a_75_anios": "EDAD_TRAMO >= 75", # Mayores a 75 anios
}

In [10]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados_fonasa = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR_FONASA, query_strings, "CUENTA_BENEFICIARIOS", "FONASA"
)

# Une todos los resultaods en un unico DataFrame
poblaciones_estratos_calculados_fonasa = pd.concat(poblaciones_estratos_calculados_fonasa)
poblaciones_estratos_calculados_fonasa = poblaciones_estratos_calculados_fonasa.reset_index(
    names=["Edad Incidencia", "Estrato"]
)

## Obtener porcentaje de población FONASA total

En este caso, se quiere saber el % de población FONASA a nivel Nacional. Esto se utilizará para obtener los casos FONASA al 2035.

In [14]:
poblacion_INE_nacional_2018_y_2022 = poblaciones_estratos_calculados_ine.query(
    "`Edad Incidencia` == 'todos' and Estrato == 'Pais'"
)[COLUMNAS_POBLACION_FONASA]

poblacion_FONASA_nacional_2018_y_2022 = poblaciones_estratos_calculados_fonasa.query(
    "`Edad Incidencia` == 'todos' and Estrato == 'Pais'"
)[COLUMNAS_POBLACION_FONASA]


---

En este apartado se leerán las incidencias, junto a las poblaciones respectivas donde se tienen que aplicar tales incidencias. Posteriormente, se calcularán los casos teóricos para cada problema de salud, y para cada estrato definido.

## Lectura de Incidencias por problema de salud

In [15]:
RUTA_PLANILLA_INCIDENCIAS = "../data/external/incidencias_y_prevalencias.xlsx"
COLUMNAS_A_UTILIZAR = ["Diagnostico", "Estadística", "Casos (Cada 100.000)", "Edad Incidencia"]


# Lee planilla de incidencias
incidencias = pd.read_excel(RUTA_PLANILLA_INCIDENCIAS, usecols=COLUMNAS_A_UTILIZAR)
# Convierte las incidencia a numeros e imputa las faltantes como NaN
incidencias["Casos (Cada 100.000)"] = pd.to_numeric(
    incidencias["Casos (Cada 100.000)"], errors="coerce"
)

# Obtiene la fraccion de la incidencia, para multiplicar con la poblacion directamente
incidencias["rate_incidencia"] = incidencias["Casos (Cada 100.000)"] / 100000

## Estimacion de casos con INE

In [16]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
union_poblaciones_e_incidencias_ine = incidencias.merge(
    poblaciones_estratos_calculados_ine, how="left", on="Edad Incidencia"
)

# Multiplica la incidencia por las poblaciones
union_poblaciones_e_incidencias_ine[COLUMNAS_POBLACION_INE] = union_poblaciones_e_incidencias_ine[
    COLUMNAS_POBLACION_INE
].multiply(union_poblaciones_e_incidencias_ine["rate_incidencia"], axis="index")

# Elimina rate de incidencia para claridad
union_poblaciones_e_incidencias_ine = union_poblaciones_e_incidencias_ine.drop(
    columns="rate_incidencia"
)

## Estimacion de casos con FONASA

In [17]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
union_poblaciones_e_incidencias_fonasa = incidencias.merge(
    poblaciones_estratos_calculados_fonasa, how="left", on="Edad Incidencia"
)

# Multiplica la incidencia por las poblaciones
union_poblaciones_e_incidencias_fonasa[COLUMNAS_POBLACION_FONASA] = (
    union_poblaciones_e_incidencias_fonasa[COLUMNAS_POBLACION_FONASA].multiply(
        union_poblaciones_e_incidencias_fonasa["rate_incidencia"], axis="index"
    )
)

# Elimina rate de incidencia para claridad
union_poblaciones_e_incidencias_fonasa = union_poblaciones_e_incidencias_fonasa.drop(
    columns="rate_incidencia"
)

In [18]:
with pd.ExcelWriter("../data/interim/casos_teoricos_diagnosticos.xlsx") as writer:
    union_poblaciones_e_incidencias_ine.to_excel(
        writer, sheet_name="casos_teoricos_INE", index=False
    )
    poblaciones_estratos_calculados_ine.to_excel(writer, sheet_name="poblacion_INE", index=False)

    union_poblaciones_e_incidencias_fonasa.to_excel(
        writer, sheet_name="casos_teoricos_FONASA", index=False
    )
    poblaciones_estratos_calculados_fonasa.to_excel(
        writer, sheet_name="poblacion_FONASA", index=False
    )